### <Center> Лабораторна робота №6. <br> Ідентифікація користувача за допомогою логістичної регресії

In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

Спочатку налаштуємо доступ до даних на google drive (якщо ви відкриваєте блокнот в google colab, а не на PC) шляхом монтування google drive

In [ ]:

#Uncomment next sections if use in google collab
!pip install google-colab
!pip install pydrive


In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:

downloaded = drive.CreateFile({'id':"1xsM59ar2cqvoC2wNpzT1Yd1bTD8Cyd6O"})
downloaded.GetContentFile('train_sessions.csv')

downloaded = drive.CreateFile({'id':"13MdleKzHiT4bkUzDsUrON37UXa4gYheY"})
downloaded.GetContentFile('test_sessions.csv')

downloaded = drive.CreateFile({'id':"1fBtf851dwyk9v2gVXIAtzq3JikHwiHig"})
downloaded.GetContentFile('site_dic.pkl')


### 1. Завантаження і перетворення даних
Дані можна самостійно завантажити за посиланням [сторінка](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). Однак можна цього не робити, оскільки дані вже завантажені для проведення лабораторної роботи

In [7]:
# завантажимо навчальну і тестову вибірки
train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('test_sessions.csv',
                      index_col='session_id')

# приведемо колонку time1, ..., time10 до часового формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# відсортуємо дані за часом
train_df = train_df.sort_values(by='time1')

# подивимося на заголовок навчальної вибірки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,945.0,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,946.0,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,952.0,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В навчальній вибірці містяться наступні ознаки:
    - site1 – індекс першого відвідування сайту в сесії
    - time1 – час відвідування першого сайту в сесії
    - ...
    - site10 – індекс 10-го відвідування сайту в сесії
    - time10 – час відвідування 10-го сайту в сесії
    - target – цільова змінна, 1 для сесій Еліс, 0 для сесій інших користувачів
    
Сесії користувачів виділені таким чимном, щоб вони не можут бути довші півгодини чи 10 сайтів. Тобто сесія вважається закінченою або коли користувач відвідав 10 сайтів підряд або коли сесія зайняла за часом більше 30 хвилин.

В таблиці зустрічаються пропущені значення, це значить, що сесія містить менше, ніж 10 сайтів. Замінимо пропущені значення нулями і приведемо ознаки до цільового типу. Також заванатажимо словник сайтів і подивимося, як він виглядає:

In [8]:
# приведемо колонки site1, ..., site10 до цілочислового формату і замінимо пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# завантажимо словник сайтів
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словника сайтів
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всього сайтів:', sites_dict_df.shape[0])
sites_dict_df.head()

всього сайтів: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Виділимо цільову змінну і об'єднаємо вибірки, щоб разом привести їх до розрідженого формату.

In [9]:
# наша цільова змінна
y_train = train_df['target']

# об'єднана таблиця вхідних даних
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# індекс, за яким будемо відокремлювати навчальну вибірку від тестової
idx_split = train_df.shape[0]

Для самої першої моделі ми використовуємо лише відвідувані сайти в сесіях (але не будемо звертати увагу на часові ознаки). В основі такого вибору даних для моделей лежить така ідея: * у Еліс є свої улюблені сайти, і якщо ви ще побачите ці сайти в сесіях, тим вище ймовірність, що це сесія Еліс і навпаки. *

Підготуємо дані, з усієї таблиці виберемо лише ознаки `site1, site2, ..., site10`. Нагадуємо, що пропущені значення замінені нулем. Ось як виглядатимуть перші рядки таблиць:

In [10]:
# таблиця з індексами відвіданих сайтів в сесії
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:


Сесії представляють собою послідовність індексів сайтів і дані в такому вигляді невдалі для лінійних методів. Відповідно до нашої гіпотези (у Еліс є улюблені сайти) необхідно перетворити цю таблицю таким чином, щоб кожен можливий веб-сайт відповідав своєму окремому призначенню (колонка), а його значення зростало за кількістю відвідувачів цього веб-сайту в сесіях. Це робиться в два рядки:

In [11]:
# послідовність з індексами
sites_flatten = full_sites.values.flatten()

# шукана матриця
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

Ще один плюс використання розріджених матриць у тому, що для них є спеціальні реалізації як матричних операцій, так і алгоритми машинного навчання, що дозволяє істотно прискорити операції за рахунок особливостей структур даних. Це стосується і логістичної регресії. Ось тепер у нас все готове для побудови наших перших моделей.

### 2. Побудова першої моделі

Отже, у нас є алгоритм та дані для нього, побудуйте нашу першу модель, використовуючи реалізацію [логістичної регресії] (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) з пакета ` sklearn` з параметрами за замовчуванням. Перші 90% даних будемо використовувати для навчання (навчальна вибірка, відсортована за часом), а також 10% для перевірки якості (validation).

** Напишіть просту функцію, яка поверне якість моделей на вкладеній вибірці, і навчіть наш перший класифікатор **.

In [12]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – вибірка
    ratio – у якому співвідношенні поділити вибірку
    C, seed – коефіцієнт регуляризації і random_state 
              логістичної регресії
    '''
    # split train dataset
    split_index = round(X.shape[0] * ratio)
    train_X = X[:split_index, :]
    test_X = X[split_index:, :]
    train_Y = y[:split_index]
    test_Y = y[split_index:]
    # fit the model
    model = LogisticRegression(C=C, random_state=seed, n_jobs=-1).fit(train_X, train_Y)
    # calc roc auc score
    class_probs = model.predict_proba(test_X)[:, 1]
    return roc_auc_score(test_Y, class_probs)

**Подивіться, який отримано ROC AUC на відкладеній вибірці.**

In [13]:
train_matrix = full_sites_sparse[:idx_split, :]
get_auc_lr_valid(train_matrix, y_train)

0.919978947171379

Будем вважати цю модель нашою першою відправною точкою (базовий рівень). Для побудови моделей для прогнозування на тестовій вибірці ** необхідно навчити модель заново вже на всій навчальній вибірці ** (покищо наша модель навчилася лише на частині даних), що підвищує її узагальнюючу здатність:

In [14]:
# функція для запису прогнозів в файлі
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Навчіть модель на всій вибірці, зробіть прогноз для тестової вибірки і покажіть результат**.

In [15]:
test_matrix = full_sites_sparse[idx_split:, :]
model = LogisticRegression(n_jobs=-1).fit(train_matrix, y_train)

In [16]:
# indices where target is 1
predict_classes = model.predict(test_matrix)
np.where(predict_classes == 1)

(array([  425,   708,   808,  1296,  1395,  1489,  1704,  1977,  2364,
         2458,  2622,  2900,  3945,  4300,  5246,  5475,  6530,  6968,
         7245,  9686, 10109, 10347, 12334, 12512, 12684, 13269, 13986,
        14405, 14459, 14590, 15027, 15136, 15268, 15340, 15937, 16066,
        16238, 17392, 17931, 18848, 19076, 19572, 19842, 20324, 20716,
        20782, 20866, 22612, 23117, 24976, 25303, 25372, 25422, 25439,
        26377, 27512, 28453, 29330, 29489, 30918, 32113, 32458, 32466,
        33068, 33304, 33910, 34295, 34484, 34639, 35420, 36707, 37075,
        37272, 38242, 38652, 39563, 40529, 41098, 41117, 41822, 41824,
        41953, 42031, 42944, 42955, 43103, 43172, 43512, 43543, 43635,
        46300, 46511, 46885, 47891, 48084, 48662, 49309, 50632, 51062,
        51245, 52819, 53103, 53427, 53859, 54518, 54591, 55100, 55124,
        56237, 57230, 57529, 57645, 57711, 57961, 58662, 58777, 59582,
        59775, 59777, 61519, 61703, 62485, 64257, 64713, 65235, 65348,
      

In [17]:
# probability of classes
classes_prob = model.predict_proba(test_matrix)
classes_prob[:20]

array([[9.97780236e-01, 2.21976363e-03],
       [9.99999997e-01, 2.51896229e-09],
       [9.99999994e-01, 6.16027591e-09],
       [9.99999987e-01, 1.32268970e-08],
       [9.99972709e-01, 2.72906695e-05],
       [9.99848820e-01, 1.51180478e-04],
       [9.99557621e-01, 4.42378701e-04],
       [9.99898755e-01, 1.01245078e-04],
       [9.99222544e-01, 7.77456105e-04],
       [8.95049220e-01, 1.04950780e-01],
       [9.99976202e-01, 2.37984922e-05],
       [9.99904476e-01, 9.55242625e-05],
       [9.99713261e-01, 2.86739456e-04],
       [6.41011349e-01, 3.58988651e-01],
       [9.99965289e-01, 3.47106166e-05],
       [9.96168730e-01, 3.83127029e-03],
       [9.84580165e-01, 1.54198348e-02],
       [9.99971672e-01, 2.83279784e-05],
       [9.99215706e-01, 7.84294328e-04],
       [9.99999881e-01, 1.18946686e-07]])

### 3. Покращення моделі, побудова нових ознак

Створіть таку ознаку, яка буде представлят собою число формату ГГГГММ від тої дати, коли відбувалась сесія, наприклад 201407 -- 2014 рік і 7 месяц. Таким чином, ми будемо враховувати помісячний [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь період наданих даних.

In [41]:
# we can be exactly sure in existing of time at 'time1' column 
extra_features_df = pd.DataFrame(index=full_df.index)
extra_features_df["start_month"] = full_df["time1"].apply(lambda time: time.year * 100 + time.month)
extra_features_df.head()

,start_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


Додайте нову ознаку, попередньо нормалізуючи її за допомогою `StandardScaler`, і знову підрахуйте ROC AUC на відкладеній вибірці.

In [43]:
start_month = extra_features_df["start_month"].values[:idx_split].reshape(-1, 1)
normalized_session_date = StandardScaler().fit_transform(start_month)
train_matrix_with_session_date = csr_matrix(hstack((train_matrix, normalized_session_date)))
get_auc_lr_valid(train_matrix_with_session_date, y_train)

0.9196563464631973

** Додайте дві нові ознаки: start_hour і morning. **

Ознака `start_hour` - це час у якому почалася сесія (від 0 до 23), а бінарна оознака` morning` рівна 1, якщо сесія почалася вранці і 0, якщо сесія почалася пізніше (будемо вважати, що це ранок, якщо `start_hour рівний` 11 або менше).

** Підрахуйте RUC AUC на відкладеній вибірці для вибірки з: **
- сайтами, `start_month` і` start_hour`
- сайтами, `start_month` і` morning`
- сайтами, `start_month`,` start_hour` і `morning`

In [44]:
extra_features_df["start_hour"] = full_df["time1"].apply(lambda time: time.hour)
extra_features_df["morning"] = full_df["time1"].apply(lambda time: 1 if time.hour <= 11 else 0)
extra_features_df.head()

,start_month,start_hour,morning
session_id,,,
21669,201301,8,1
54843,201301,8,1
77292,201301,8,1
114021,201301,8,1
146670,201301,8,1


In [26]:
# Not to repeat code we will write functions for normalizing features and making train matrix
def get_normalized_values(features):
    data = np.array(features)
    if len(data.shape) <= 1:
        data = features.reshape(-1, 1)
    normalized_features = StandardScaler().fit_transform(data)
    return normalized_features

In [48]:
def get_auc_lr_valid_with_extra_features(features):
    normalized_data = get_normalized_values(features.values[:idx_split, :])
    matrix_with_extra_features = csr_matrix(hstack((train_matrix, normalized_data)))
    roc_auc_estimate = get_auc_lr_valid(matrix_with_extra_features, y_train)
    return roc_auc_estimate

In [49]:
get_auc_lr_valid_with_extra_features(extra_features_df[["start_month", "start_hour"]])

0.9572375929307422

In [50]:
get_auc_lr_valid_with_extra_features(extra_features_df[["start_month", "morning"]])

0.9481406152631044

In [30]:
get_auc_lr_valid_with_extra_features(extra_features_df[["start_month", "morning", "start_hour"]])

0.958901318904009

### 4. Підбір коефіцієнта регуляризації

Отже, ми ввели ознаки, які покращують якість нашої моделі у порівнянні з першим бейслайном. Чи можемо ми домогтися більшого значення метрики? Після того, як ми сформували навчальну та тестову вибірки, майже завжди має сенс підібрати оптимальні гіперпараметри - характеристики моделі, які не змінюються під час навчання. Наприклад, ви вивчали вирішальні дерева, глибина дерева це гіперпараметр, а ознака, за якому відбувається розгалуження і її значення - ні. У використовуваної нами логістичної регресії ваги кожної ознаки змінюються і під час навчання знаходяться їх оптимальні значення, а коефіцієнт регуляризації залишається постійним. Це той гіперпараметр, який ми зараз будемо оптимізувати.

Порахуйте якість на відкладеній вибірці з коефіцієнтом регуляризації, який за замовчуванням `C = 1 ':

In [31]:
get_auc_lr_valid(train_matrix, y_train)

0.9199789471713791

Постараємося побити цей результат за рахунок оптимізації коефіцієнта регуляризації. Візьмемо набір можливих значень C і для кожного з них порахуємо значення метрики на відкладеної вибірці.

Знайдіть `C` з` np.logspace (-3, 1, 10) `, при якому ROC AUC на відкладеної вибірці максимальний.

In [38]:
def compare(c_value):
    if c_value < 0:
      return 0
    auc_value = get_auc_lr_valid(train_matrix, y_train, C=c_value)
    print(f"{auc_value} auc roc estimation with {c_value} value")
    return auc_value

In [40]:
C_values = np.logspace(-1, 3, 10)
optimal_C = max(C_values, key=compare)
print(f"{optimal_C} is the best C value")

0.920034953190441 auc roc estimation with 0.1 value
0.9200624278413017 auc roc estimation with 0.2782559402207124 value
0.9194649296648938 auc roc estimation with 0.774263682681127 value
0.9208957249443261 auc roc estimation with 2.1544346900318834 value
0.9214924683226333 auc roc estimation with 5.994842503189409 value
0.9207619747209059 auc roc estimation with 16.68100537200059 value
0.9206528309155642 auc roc estimation with 46.41588833612777 value
0.9208403227637446 auc roc estimation with 129.15496650148827 value
0.9205705579226025 auc roc estimation with 359.38136638046257 value
0.9201767042737272 auc roc estimation with 1000.0 value
5.994842503189409 is the best C value


Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

Нарешті, навчіть модель зі знайденим оптимальним значенням коефіцієнта регуляризації і з побудованими ознаками `start_hour`,` start_month` і `morning`. 

In [53]:

def make_matrix(rows_slice):
    extra_features = extra_features_df[["start_month", "morning", "start_hour"]].values[rows_slice, :]
    extra_normalized_features = get_normalized_values(extra_features)
    final_matrix = csr_matrix(hstack((full_sites_sparse[rows_slice, :], extra_normalized_features)))
    return final_matrix

In [55]:
# preparing data
final_train_matrix = make_matrix(slice(None, idx_split, 1))
final_model = model = LogisticRegression(C=optimal_C, n_jobs=-1).fit(final_train_matrix, y_train)

In [58]:
final_test_matrix = make_matrix(slice(idx_split, None, 1))
y_predicted = final_model.predict(final_test_matrix)
write_to_submission_file(y_predicted, "predicted_by_final_model.csv")
np.where(y_predicted == 1)

(array([   44,    83,   346,   425,   708,   808,  1182,  1296,  1395,
         1489,  1704,  1849,  1977,  2364,  2390,  2458,  2622,  2881,
         2900,  3640,  3945,  4300,  4561,  4919,  5246,  5251,  5331,
         5939,  6246,  6288,  6530,  6681,  6968,  7245,  7888,  7969,
         7996,  8515,  8656,  9097,  9686,  9982, 10109, 10199, 10347,
        10614, 12334, 12512, 12684, 13111, 13269, 13336, 13901, 13986,
        14405, 14459, 14590, 14685, 15027, 15305, 15328, 15340, 15464,
        15937, 16238, 16674, 17018, 17062, 17392, 17749, 17931, 18726,
        18848, 19148, 19572, 19842, 19967, 20782, 20866, 20892, 21070,
        22469, 22612, 23117, 24897, 24976, 25303, 25372, 25412, 25422,
        25439, 25774, 25847, 26265, 26377, 26596, 27512, 27682, 27776,
        28453, 29037, 29082, 29330, 29360, 29489, 30558, 30918, 31801,
        32113, 32423, 32458, 32466, 32618, 32631, 33068, 33169, 33304,
        33488, 33910, 34295, 34484, 34639, 35094, 35299, 35420, 35787,
      

In [60]:
final_class_probs = final_model.predict_proba(final_test_matrix)[:, 1]
write_to_submission_file(final_class_probs, "final_classes_probabilities.csv")
final_class_probs[:20]

array([4.76908432e-06, 3.84055536e-11, 2.90066332e-17, 1.55534571e-16,
       3.42386938e-06, 2.46774599e-08, 1.02981546e-04, 4.70239939e-06,
       2.14222677e-06, 1.08616756e-03, 1.29382256e-07, 3.58572331e-08,
       5.43380362e-08, 8.05489179e-03, 4.21162998e-07, 5.10537814e-05,
       3.13869354e-02, 1.30589908e-09, 5.34596407e-06, 1.94798196e-11])